In [1]:
# minimal module imports
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayer
from arcgis.geometry import Geometry
import pandas as pd
import os
import importlib

# import arcpy if available
if importlib.util.find_spec("arcpy") is not None:
    import arcpy
    
import sys
sys.path.append('../src')

app_id = 'c9fa4c15-b9ab-46d4-bcd9-a3eb33dd0ff8'
hash_token = '919745fda981ddccb3ffc4f07863480e624cfa3d'
bg_pop_cent_path = 'D:\\projects\\geoai-retail\\data\\interim\\interim.gdb\\bg_pop_cent'
coffee_path = 'D:\\projects\\geoai-retail\\data\\raw\\raw.gdb\\coffee'

from inrix import TAZ
taz = TAZ(app_id, hash_token)

In [2]:
df_pop_cent = GeoAccessor.from_featureclass(coffee_path)

In [3]:
row = df_pop_cent.iloc[0]
geom = row.SHAPE
locnum = row.LOCNUM
print(row.LOCNUM)
print(geom.x, geom.y)

413963145
-122.99270000035045 44.77610000019564


In [6]:
taz.get_trip_destination_spatial_dataframe(geom.y, geom.x, '100m')

,deviceId,endDateTime,endLoc,endpointType,providerHash,providerType,startDateTime,startLoc,tripDistanceMeters,tripId,tripMeanSpeedKPH,travelTimeMinutes,travelDistanceMiles,SHAPE
0,03a2bf9941a4c4e24b3b598f9e8b8310,2018-02-10T01:15:02.000Z,"44.775894,-122.993318",0,f4f6dce2f3a0f9dada0c2b5b66452017,consumer,2018-02-10T00:50:00.000Z,"44.87728,-123.026415",21343,60a4c90d2b7b4f5f7416b5d1b334c2d9,51,25.033333,13.261921,"{'x': -123.026415, 'y': 44.87728, 'spatialRefe..."


In [4]:
def get_dest_df(geom, locnum):
    dest_df = taz.get_trip_destination_spatial_dataframe(geom.y, geom.x, '100m')
    if dest_df is not None:
        dest_df = dest_df[['travelDistanceMiles', 'travelTimeMinutes', 'SHAPE']].copy()
        dest_df.columns = ['travel_distance_miles', 'travel_time_minutes', 'SHAPE']
        dest_df['store_locnum'] = locnum
        return dest_df
    else:
        return None

In [5]:
df = get_dest_df(geom, locnum)
df

,travel_distance_miles,travel_time_minutes,SHAPE,store_locnum
0,13.261921,25.033333,"{'x': -123.026415, 'y': 44.87728, 'spatialRefe...",413963145


In [6]:
%%time
for idx, (locnum, geom) in df_pop_cent[['LOCNUM', 'SHAPE']].iterrows():
    new_df = get_dest_df(geom, locnum)
    if new_df is not None:
        if idx == 0:
            dest_df = new_df
        else:
            dest_df = dest_df.append(new_df)
    dest_df.SHAPE = dest_df

Wall time: 13min 15s


In [7]:
dest_df.head()

,travel_distance_miles,travel_time_minutes,SHAPE,store_locnum
0,13.261921,25.033333,"{'x': -123.026415, 'y': 44.87728, 'spatialRefe...",413963145
0,8.260506,12.466667,"{'x': -123.150157, 'y': 44.933511, 'spatialRef...",244210043
1,2.013863,8.750000,"{'x': -123.312642, 'y': 44.929472, 'spatialRef...",244210043
2,0.547428,7.516667,"{'x': -123.30801, 'y': 44.9299, 'spatialRefere...",244210043
3,0.133595,3.100000,"{'x': -123.315767, 'y': 44.922983, 'spatialRef...",244210043


In [16]:
dest_df.spatial.set_geometry('SHAPE')
dest_df.spatial.to_featureclass('D:\\projects\\geoai-retail\\data\\interim\\interim.gdb\\trips_raw')

KeyError: 'spatialReference'

In [17]:
dest_df.to_csv('raw_trips.csv')

In [13]:
dest_df['x'] = dest_df.SHAPE.apply(lambda val: val['x'])
dest_df.head()

KeyError: ('x', 'occurred at index travel_distance_miles')

In [ ]:
dest_df['y'] = dest_df.SHAPE.apply(lambda val: val.y)
dest_df.head()

In [ ]:
dest_df.reset_index(drop=True, inplace=True)
dest_df.head()

In [15]:
dest_df.spatial.validate()

True